In [ ]:
import yfinance as yf
import pandas as pd 
import numpy as np
from datetime import date
import os 
import math
def cci(data,period):
    data_c = []
    data_h = []
    data_l = []
    for i in range(0,len(data['Close'])):
        data_c += list(data['Close'][i:i+1])
        data_h += list(data['High'][i:i+1])
        data_l += list(data['Low'][i:i+1])
    tp_list = []
    for i in range(0,(len(data['Close']))):
        tp = (data_c[i] + data_h[i] + data_l[i])/3
        tp_list.append(tp)
    sma_tp_list = []
    for i in range(0,len(tp_list)-period+1):
        sma_tp_list.append((sum(tp_list[i:period+i])/period))
    md_list = []
    for i in range(0,len(tp_list)-period+1):
        tp_n_list = tp_list[i:period+i]
        md = 0
        for j in tp_n_list:
            md += abs(j - sma_tp_list[i])
        md_list.append((md/period))
    cci_list = []
    for i in range(0,len(tp_list)-period+1):
        cci_list.append((tp_list[period+i-1] - sma_tp_list[i])/(0.015*md_list[i]))
    cci_list = [float(x) for x in cci_list]
    cci_list = [float('nan')] * (period-1) + cci_list
    return cci_list


def rsi(data,period,smoothing):
    up_list = []
    down_list = []
    for i in range(0,len(data['Close'])-1):
        change = data['Close'].iloc[i+1] - data['Close'].iloc[i]
        if change>=0:
            up_list.append(change)
            down_list.append(0)
        else:
            up_list.append(0)
            down_list.append(-change)

    avg_gain_list = []
    avg_loss_list = []
    avg_gain_list.append(sum(up_list[0:period])/period)
    avg_loss_list.append(sum(down_list[0:period])/period)
    for i in range(1,len(data['Close'])-period):
        next_gain = (avg_gain_list[i-1]*(period-1) + up_list[period+i-1])/period
        avg_gain_list.append(next_gain)
        next_loss = (avg_loss_list[i-1]*(period-1) + down_list[period+i-1])/period
        avg_loss_list.append(next_loss)
    rsi_list = []
    for i in range(0,len(avg_gain_list)):
        rs= avg_gain_list[i]/avg_loss_list[i]
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    rsi_list = [float(x) for x in rsi_list]
    rsi_list = [float('nan')] * (period) + rsi_list
    return rsi_list,pd.Series(rsi_list).rolling(window=smoothing).mean().tolist()


def ema(data,period):
    k = 2/(period+1)
    ema_list=[]
    ema_list.append(sum(data['Close'][0:period])/period)
    for i in range(1,len(data['Close'])-period+1):
        next_ema = (data['Close'].iloc[period+i-1]*k)+(ema_list[i-1]*(1-k))
        ema_list.append(next_ema)
    return ema_list


def macd(data,period1,period2,slsf):
    ema5 = ema(data,period1)
    ema35 = ema(data,period2)
    ema5trunc = ema5[period2-period1::]
    macds = {}
    macd_r = [a - b for a, b in zip(ema5trunc, ema35)]
    macds = {"Close":macd_r}
    macdf = pd.DataFrame(macds)
    signal = ema(macdf,slsf)
    histogram = [a - b for a, b in zip(macd_r[slsf-1::], signal)]
   # print("Signal:",signal[-1])
    #print("MACD:",macd_r[-1])
    histogram = [float(x) for x in histogram]
    histogram = [float('nan')] * (period2+slsf-2) + histogram
    print("histogram:",histogram[-3:])
    return histogram


def stochastics(data,k_l,k_s,d_s):
    k_fast_list = []
    for i in range(0,len(data['Close'])-k_l+1):
        hh = max(data['High'][i:k_l+i])
        ll = min(data['Low'][i:k_l+i])
        k = ((data['Close'].iloc[k_l+i-1] - ll)/(hh-ll))*100
        k_fast_list.append(k)
    k_list = []
    for i in range(0,len(k_fast_list)-k_s+1):
        k_list.append(sum(k_fast_list[i:k_s+i])/k_s)
    d_list = []
    for i in range(0,len(k_list)-d_s+1):
        d_list.append(sum(k_list[i:d_s+i])/d_s)
    k_list = [float('nan')] * (k_l+k_s-2) + k_list
    d_list = [float('nan')] * (k_l+k_s+d_s-3) + d_list
    k_list = [float(x) for x in k_list]
    d_list = [float(x) for x in d_list]
    return k_list,d_list

def sma(data,period):
    sma = data['Close'].rolling(window=period).mean()
    return sma

def stock_ochl(data,stock):
    stock_price = {}
    stock_price['Open'] = data['Open'][stock]
    stock_price['Close'] = data['Close'][stock]
    stock_price['High'] = data['High'][stock]
    stock_price['Low'] = data['Low'][stock]
    return stock_price

def oc_move(data):
    oc_move = data['Close']-data['Open']
    oc_move = oc_move.shift(-1)
    return oc_move

def log_cc_move(data):
    log_cc_move = [float('nan')]
    for i in range(len(data['Close'])-1):
        log_cc_move.append(math.log(data['Close'][i+1])-math.log(data['Close'][i]))
    return log_cc_move

symbol = 'NVDA'
data = yf.download(symbol, period="5y", interval="1d",auto_adjust=False)

def indicators_dataset(data,stock_price):
    k_list, d_list = stochastics(stock_price,12,3,3)
    data['KStochastic'] = k_list
    data['DStochastic'] = d_list
    data['SMA10'] = sma(data,10)
    data['SMA20'] = sma(data,20)
    data['SMA30'] = sma(data,30)
    data['SMA200'] = sma(data,200)
    data['macd'] = macd(stock_price,5,35,5)
    data['cci'] = cci(stock_price,90)
    data['rsi'],data['rsim'] = rsi(stock_price,21,14)
    data['log_cc_move'] = log_cc_move(stock_price)
    #199 if including SMA200 or just 89 if max is CCI, may need customization if using different sizes of SMA 
    data = data[199:]
    data.columns = data.columns.droplevel(1)
    data.drop(['Adj Close','Close','High','Low','Open','Volume'], axis=1, inplace=True)
    return data
def log_move_dataset(data,stock_price,shift):
    logccmove = log_cc_move(stock_price)
    data['shift1'] = [float('nan')]+logccmove[:-1]
    for i in range(2,shift+1):
        data['shift'+str(i)] = [float('nan')] + data['shift'+str(i-1)][:-1].tolist()
    data['log_cc_move'] = log_cc_move(stock_price)
    data = data[1+shift:]
    data.columns = data.columns.droplevel(1)
    data.drop(['Adj Close','Close','High','Low','Open','Volume'], axis=1, inplace=True)
    return data
stock_price = stock_ochl(data,symbol)
shift = 20

#comment out which data you want for the dataset (if you want an indicators dataset or a log close move dataset)
data = log_move_dataset(data,stock_price,shift)
#data = indicators_dataset(data,stock_price)

main_folder = str(os.getcwd())+'/data/stocks'
os.makedirs(main_folder, exist_ok=True)
now = date.today()
path = main_folder+'/'+symbol+str(now.year)+'-'+str(now.month)+'-'+str(now.day)+'-'+str(shift)+'.csv'
print(path)
data.to_csv(path, index=False)



: 